In [9]:
import glob

import pandas as pd
from pathlib import Path

from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator, rdMolDescriptors

In [10]:
OUTPUT_DIRECTORY = '../../data/paired_spectra/casmi2022/spec_files'
Path(OUTPUT_DIRECTORY).mkdir(exist_ok=True)

In [3]:
df = pd.read_pickle('../../data/paired_spectra/casmi2022/processed_massformer/spec_df.pkl')
print(len(df))
df.head()

229


,casmi_id,ion_mode,priority,smiles,prec_type,rt,prec_mz,spec_type,peaks,ace,nce,inst_type,frag_mode,res,spec_id,group_id,spec_id_old,mol_id_old,mol_id
0,3,P,True,COc1ccc(-c2oc3c(CC=C(C)C)c(O)cc(O)c3c(=O)c2OC2...,[M+H]+,5.5959,719.253867,MS2,"[(50.65100861, 3705.4970703125), (52.45254517,...",NaN,NaN,FT,HCD,4,0,2,2,2,1196787
1,4,P,True,CC(C)CC(O)C(=O)NC(C(=O)OC(C(=O)NC(CC(C)C)C(=O)...,[M+Na]+,5.7983,467.272377,MS2,"[(51.05776978, 2370.4174804688), (51.91718674,...",NaN,NaN,FT,HCD,4,1,3,3,3,1196788
2,5,P,True,CC(=O)OC12CC(C)C(O)C1C=C(C)C(OC(=O)CCC(=O)O)CC...,[M+Na]+,5.5829,499.229626,MS2,"[(51.24491882, 2307.3256835938), (53.03874207,...",NaN,NaN,FT,HCD,4,2,4,4,4,1196789
3,9,P,True,CN(C)CCc1c[nH]c2ccc(CS(=O)(=O)N3CCCC3)cc12,[M+H]+,3.4133,336.173416,MS2,"[(50.28293991, 49982.94921875), (50.28532791, ...",NaN,NaN,FT,HCD,4,3,8,8,8,1196790
4,10,P,True,Cn1c(CNc2ccc(C(=N)N)cc2)nc2cc(C(=O)N(CCC(=O)O)...,[M+H]+,2.2642,472.208380,MS2,"[(53.03887939, 6174.09765625), (53.56915283, 5...",NaN,NaN,FT,HCD,4,4,9,9,9,1196791


In [4]:
hydrogen_adduct = df[df['prec_type'] == '[M+H]+']
len(hydrogen_adduct)

170

In [5]:
hydrogen_adduct['mol'] = hydrogen_adduct['smiles'].apply(Chem.MolFromSmiles)

hydrogen_adduct['inchikey'] = hydrogen_adduct['mol'].apply(Chem.MolToInchiKey)

hydrogen_adduct['formula'] = hydrogen_adduct['mol'].apply(rdMolDescriptors.CalcMolFormula)

mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2,fpSize=4096)
hydrogen_adduct['fingerprint'] = hydrogen_adduct['mol'].apply(mfpgen.GetFingerprint)
hydrogen_adduct.head()

/var/folders/n5/t_ds6vws3ns1lyk9130hfx180000gn/T/ipykernel_17783/2049023019.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hydrogen_adduct['mol'] = hydrogen_adduct['smiles'].apply(Chem.MolFromSmiles)
/var/folders/n5/t_ds6vws3ns1lyk9130hfx180000gn/T/ipykernel_17783/2049023019.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hydrogen_adduct['inchikey'] = hydrogen_adduct['mol'].apply(Chem.MolToInchiKey)
/var/folders/n5/t_ds6vws3ns1lyk9130hfx180000gn/T/ipykernel_17783/2049023019.py:5: SettingWithCopyWar

,casmi_id,ion_mode,priority,smiles,prec_type,rt,prec_mz,spec_type,peaks,ace,...,res,spec_id,group_id,spec_id_old,mol_id_old,mol_id,mol,inchikey,formula,fingerprint
0,3,P,True,COc1ccc(-c2oc3c(CC=C(C)C)c(O)cc(O)c3c(=O)c2OC2...,[M+H]+,5.5959,719.253867,MS2,"[(50.65100861, 3705.4970703125), (52.45254517,...",NaN,...,4,0,2,2,2,1196787,<rdkit.Chem.rdchem.Mol object at 0x157872dc0>,UYALDZZEAZIEME-UHFFFAOYSA-N,C35H42O16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,9,P,True,CN(C)CCc1c[nH]c2ccc(CS(=O)(=O)N3CCCC3)cc12,[M+H]+,3.4133,336.173416,MS2,"[(50.28293991, 49982.94921875), (50.28532791, ...",NaN,...,4,3,8,8,8,1196790,<rdkit.Chem.rdchem.Mol object at 0x157872ca0>,WKEMJKQOLOHJLZ-UHFFFAOYSA-N,C17H25N3O2S,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,10,P,True,Cn1c(CNc2ccc(C(=N)N)cc2)nc2cc(C(=O)N(CCC(=O)O)...,[M+H]+,2.2642,472.208380,MS2,"[(53.03887939, 6174.09765625), (53.56915283, 5...",NaN,...,4,4,9,9,9,1196791,<rdkit.Chem.rdchem.Mol object at 0x1578a0940>,YBSJFWOBGCMAKL-UHFFFAOYSA-N,C25H25N7O3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,11,P,True,CCCC(=O)OCC(OC(=O)CCC)C(OC(=O)CCC)C(Cn1c2nc(=O...,[M+H]+,6.0879,657.311672,MS2,"[(55.05422974, 4195.5727539063), (55.98180008,...",NaN,...,4,5,10,10,10,1196792,<rdkit.Chem.rdchem.Mol object at 0x1578a0580>,MJNIWUJSIGSWKK-UHFFFAOYSA-N,C33H44N4O10,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,15,P,True,CC1CCC2(C(=O)O)CCC3(C)C(=CCC4C5(C)CC(O)C(O)C(C...,[M+H]+,4.8524,519.331159,MS2,"[(58.7617836, 2441.0568847656), (67.05400848, ...",NaN,...,4,6,14,14,14,1196793,<rdkit.Chem.rdchem.Mol object at 0x1578a05e0>,FYGSMJAMPMZYFS-UHFFFAOYSA-N,C30H46O7,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [6]:
def compound_to_ms_file(row):
    
    with open(f'{OUTPUT_DIRECTORY}/casmi2022_{row["casmi_id"]}.ms', 'w') as f:
        
        f.write(f'>compound casmi2022_{row["casmi_id"]}\n')
        f.write(f'>formula {row["formula"]}\n')
        f.write(f'>parentmass {row["prec_mz"]}\n')
        f.write(f'>rt {row["rt"]}\n')
        f.write(f'>Ionization {row["prec_type"]}\n')
        f.write(f'>InChI None\n')
        f.write(f'>InChIKey {row["inchikey"]}\n')
        f.write(f'#smiles {row["smiles"]}\n')
        f.write(f'#scans {row["spec_id"]}\n')
        
        f.write('\n')
        f.write(f'>ms2\n')

        for peak in row['peaks']:
            f.write(f'{peak[0]} {peak[1]}\n')

In [11]:
hydrogen_adduct.apply(compound_to_ms_file, axis=1)

0      None
3      None
4      None
5      None
6      None
       ... 
224    None
225    None
226    None
227    None
228    None
Length: 170, dtype: object

In [13]:
# Create labels file for ...
ms_files = glob.glob(f'{OUTPUT_DIRECTORY}/*.ms')

df = pd.DataFrame(columns=['dataset', 'spec'])

for ms_file in ms_files:
    spec = ms_file.split('/')[-1].split('.')[0]
    df.loc[len(df)] = {'dataset': 'casmi2022', 'spec': spec}

df.to_csv('../../data/paired_spectra/casmi2022/labels.tsv', sep="\t", index=None)

In [15]:
# Create true labels file
ms_files = glob.glob(f'{OUTPUT_DIRECTORY}/*.ms')

df = pd.DataFrame(columns=['dataset', 'spec', 'name', 'ionization', 'formula', 'smiles', 'inchikey'])

for ms_file in ms_files:
    
    spec = ms_file.split('/')[-1].split('.')[0]

    spec_id = int(spec.split('_')[-1])
    row = hydrogen_adduct[hydrogen_adduct['casmi_id'] == spec_id]
    
    df.loc[len(df)] = {'dataset': 'casmi2022', 
                       'spec': spec, 
                       'name': '', 
                       'ionization': row['prec_type'].values[0], 
                       'formula': row['formula'].values[0], 
                       'smiles': row['smiles'].values[0], 
                       'inchikey': row['inchikey'].values[0]}

df.to_csv('../../data/paired_spectra/casmi2022/labels_true.tsv', sep="\t", index=None)